In [117]:
from bs4 import BeautifulSoup
from requests_html import HTMLSession
import json
import re

In [118]:
session = HTMLSession()
url = 'https://vulcanpost.com/my/'
response = session.get(url)

soup = BeautifulSoup(response.content, 'html.parser')

In [120]:
elements = soup.find_all(attrs={"class": ["elementor-icon-box-title", "elementor-heading-title"]})
articles = []

for element in elements:
  url = element.find("a")
  if url is not None and url.get("href") != 'https://vulcanpost.com/latest-stories/':
    link = url.get("href")
    title = element.text.strip()

    if re.match(r'^https://vulcanpost\.com/\d+/[\w-]+/$', link):
      title = element.text.strip()
      articles.append({
          "url": link,
          "title": title
      })
      
print(len(articles))

56


In [121]:
for article in articles:
  response = session.get(article["url"])
  article_content = BeautifulSoup(response.content, 'html.parser')
  texts = article_content.find("div", attrs={"class": "elementor-widget-theme-post-content"})
  if texts is not None:
    p_elements = texts.find_all("p")
    body = []
    for element in p_elements:
      if element is not None:
        body.append(element.text.strip())
    article["body"] = " ".join(body)
    
print(len(articles))

56


In [123]:
with open('vulcanpost-articles.json', 'w', encoding='utf-8') as json_file:
  json.dump(articles, json_file, ensure_ascii=False, indent=4)